In [ ]:
import ee
import geemap
import geopandas as gpd

# Initialize gee
geemap.ee_initialize()

In [12]:
# Settings

# Set City, available options:
'''
    Athina
    Thessaloniki
    Patra
    Irakleio
    Larisa
    Volos
    Ioannina
    Kavala
    Kalamata
'''

CITY = 'Volos'

# scaling parameter for GEE
TILES=16

# End member values were sampled near Volos LUZ for 2014 
YEAR_EM = 2014
EM_GPKG= f'./EM/em_meanValues_{YEAR_EM}_Volos.gpkg'



In [13]:
# LUZ vector , filter one City
LUZ = (ee.FeatureCollection('projects/ee-leodliakos/assets/LUZ')
.filter(ee.Filter.inList('fua_name', [CITY])))

# Date ranges for Landsat 5 & 8 timeseries
DATES_LS5=('2000-01-01','2012-01-01')
DATES_LS8=('2014-01-01','2021-01-01')

# Gee datasets for Landsat timeseries
ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") # USGS Landsat 5 Level 2, Collection 2, Tier 1
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") # USGS Landsat 8 Level 2, Collection 2, Tier 1

# LandSat 5

Preprocessing for LandSat 5 images

In [14]:
# Filter landsat5 timeseries based on dates and the extents of current City
ls5 = (ls5 
    .filterDate(*DATES_LS5) 
    .filterBounds(LUZ) )


def apply_scale_factorsL5(image):
  '''Apply scaling factors'''
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# apply on timeseries  
ls5 = ls5.map(apply_scale_factorsL5);

# 
def bitwiseExtract(input, fromBit, toBit):
  '''
  function to decode bits
  source: https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
  '''
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)



def maskL5sr(image):
  ''' function to mask Landsat 5 timeseries'''
  
  FillCloudBitMask = 1 << 0
  DilatedCloudBitMask = 1 << 1
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 <<4 
  snowsBitMask = 1 << 5
  WaterBitMask = 1 << 7
  
  # Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
    .And(bitwiseExtract(qa, 8, 9).eq(1)) \
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
    .And(qa.bitwiseAnd(snowsBitMask).eq(0)) \
    .And(qa.bitwiseAnd(FillCloudBitMask).eq(0)) \
    .And(qa.bitwiseAnd(DilatedCloudBitMask).eq(0))  
  
  # remove saturated 
  saturationMask = image.select('QA_RADSAT').eq(0);
             
  # Apply the mask
  maskedimage = image.updateMask(mask).updateMask(saturationMask)
  
  # Attach a new band, Water band
  maskedimage = maskedimage.addBands(
    ee.Image([qa.bitwiseAnd(WaterBitMask).eq(0).rename('Water')])
    ).copyProperties(image).set('system:time_start', image.get('system:time_start'));
     
  return (maskedimage)

# Apply masks in LandSat5 timeseries
landsat5FiltMasked = ls5.map(maskL5sr)

# Create a new Water Image from full timeseries based on mode reduction
water5 = landsat5FiltMasked.select('Water').reduce(ee.Reducer.mode(),TILES)


def addVariables5(image):
  '''Function to add NDVI and NDWI in Landsat 5 images
  Return the image with the added bands'''
  
  return (image
  .addBands(image.normalizedDifference(['SR_B4', 'SR_B3']) # Add an NDVI band.
            .multiply(0.9723).add(0.0235).rename('NDVI'))  # Apply regression coefficients, source: https://doi.org/10.1016/j.rse.2015.12.024
  .addBands(image.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI'))) # Add an NDWI band.
  
  
# apply the Function  
landsat5FiltMasked = landsat5FiltMasked.map(addVariables5)

# Select useful bands and rename band names identical to Landsat 8 (to merge latter)
landsat5FiltMasked= landsat5FiltMasked.select(['SR_B1', # blue
                      'SR_B2', # green
                      'SR_B3',# red
                      'SR_B4', # IR
                      'NDVI', 
                      'NDWI']).map(lambda x: x.rename(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'NDVI', 'NDWI'])) # rename, identical to Landsat8

# Function to create annual max reducer for LandSat 5
def func_apr5(year):
    """Calculate annual max for Bands"""
    date_start = ee.Date.fromYMD(year, 1, 1)
    date_end = date_start.advance(1, "year")

    image1 = (landsat5FiltMasked.filterDate(date_start, date_end) 
        .reduce(ee.Reducer.max(), TILES) 
        .set({"year": year}) 
        .set({"system:time_start": date_start.millis(), "system:time_end": date_end.millis()}))

    return(image1)


# Apply the func_apr5 function, for each year return max composite of bands
annual_max_NDVI_median_NDVI5 = ee.ImageCollection.fromImages(
    ee.List.sequence(
        ee.Date(DATES_LS5[0]).get("year").getInfo(),
        ee.Date(DATES_LS5[1]).get("year").getInfo() - 1,
    ).map(func_apr5)
)


# LandSat 8

LandSat 8 preprocessing

In [15]:
# Filter Landsat 8 timeseries based on dates and the extents of current City
ls8 = ls8 \
    .filterDate(*DATES_LS8) \
    .filterBounds(LUZ) 
    
    

def applyScaleFactors(image):
  '''Function to apply scaling factors'''
  
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
  return (image.addBands(opticalBands, None, True)
              .addBands(thermalBands, None, True))
  
# Apply rescale in the images of timeseries
ls8 = ls8.map(applyScaleFactors)



def maskL8sr(image):
  '''function to mask Landsat 8 timeseries'''
  
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  FillCloudBitMask = 1 << 0
  DilatedCloudBitMask = 1 << 1
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 <<4 
  snowsBitMask = 1 << 5
  WaterBitMask = 1 << 7
  
  # Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
    .And(bitwiseExtract(qa, 8, 9).eq(1)) \
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
    .And(qa.bitwiseAnd(snowsBitMask).eq(0)) \
    .And(qa.bitwiseAnd(FillCloudBitMask).eq(0)) \
    .And(qa.bitwiseAnd(DilatedCloudBitMask).eq(0))  

  
  # remove saturated 
  saturationMask = image.select('QA_RADSAT').eq(0);
  
           
  # Apply the mask
  maskedimage = image.updateMask(mask).updateMask(saturationMask)
  
  # Attach a new band, Water band
  maskedimage = maskedimage.addBands(
    ee.Image([qa.bitwiseAnd(WaterBitMask).eq(0).rename('Water')])
    ).copyProperties(image).set('system:time_start', image.get('system:time_start'));
    
  return (maskedimage)


# Remove clouds, snow etc.
landsat8FiltMasked = ls8.map(maskL8sr)

# Create a new Water Image from full timeseries based on mode reduction
water8 = landsat8FiltMasked.select('Water').reduce(ee.Reducer.mode(),TILES)

def addVariables(image):
  '''
  Function to add NDVI, time, and constant variables to Landsat 8 imagery.
  Return the image with the added bands
  '''
  return(image
         .addBands(image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))
         .addBands(image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')))
  
# apply the function
landsat8FiltMasked = landsat8FiltMasked.map(addVariables)

# Select useful bands
landsat8FiltMasked = landsat8FiltMasked.select(['SR_B2', #blue
                    'SR_B3',# green
                    'SR_B4', # red
                    'SR_B5', # IR
                    'NDVI', 
                    'NDWI'])

# Function to create annual max reducer for LandSat 8
def func_apr(year):
    """Calculate annual max for Bands"""
    date_start = ee.Date.fromYMD(year, 1, 1)
    date_end = date_start.advance(1, "year")

    image1 = (landsat8FiltMasked.filterDate(date_start, date_end) 
        .reduce(ee.Reducer.max(), TILES) 
        .set({"year": year}) 
        .set({"system:time_start": date_start.millis(), "system:time_end": date_end.millis()}))

    return(image1)


# for each year return max composite of bands
annual_max_NDVI_median_NDVI8 = ee.ImageCollection.fromImages(
    ee.List.sequence(
        ee.Date(DATES_LS8[0]).get("year").getInfo(),
        ee.Date(DATES_LS8[1]).get("year").getInfo() - 1,
    ).map(func_apr)
)

In [16]:
# Merge the two timeseries
annual_max_NDVI_median_NDVI = annual_max_NDVI_median_NDVI8.merge(annual_max_NDVI_median_NDVI5)


# End Members

Read data for EM.

A geopackage with precalculated samples. 

Mean values from max composites of **NDVI, NDWI, B2, B3, B4, B5** for the three end members: 
- Vegetation
- Urban
- Water

In [17]:
em_mean = gpd.read_file(EM_GPKG)
em_mean


,NDVI_max,NDWI_max,SR_B2_max,SR_B3_max,SR_B4_max,SR_B5_max,name,geometry
0,0.905461,-0.607540,0.031255,0.067272,0.064674,0.445281,Vegetation,"MULTIPOLYGON (((23.09318 39.38016, 23.10511 39..."
1,0.117699,-0.081221,0.176029,0.223522,0.256718,0.298689,Urban,"MULTIPOLYGON (((22.88583 39.37206, 22.88770 39..."
2,0.844856,0.902014,0.028305,0.035752,0.030032,0.051691,Water,"MULTIPOLYGON (((23.21669 39.36795, 23.22098 39..."


In [18]:
# get Band Names 
BANDS = annual_max_NDVI_median_NDVI.first().bandNames()

In [19]:

Vegetation = em_mean[(em_mean['name'] == 'Vegetation')][BANDS.getInfo()].values.tolist()[0]
Urban = em_mean[(em_mean['name'] == 'Urban')][BANDS.getInfo()].values.tolist()[0]
Water = em_mean[(em_mean['name'] == 'Water')][BANDS.getInfo()].values.tolist()[0]


# Spectral Unmix

In [ ]:
def func_Unmix(year):
    '''Unmix for one year'''
    
    date_start = ee.Date.fromYMD(year, 1, 1)
    date_end = date_start.advance(1, "year")

    img = annual_max_NDVI_median_NDVI.filterDate(date_start, date_end).first()  

    # BANDS = img.bandNames()
    
    # Calculate fraction
    fraction = img.unmix([Water,Urban,Vegetation], sumToOne=True, nonNegative=True) \
            .rename(['Water', 'Urban', 'Vegetation']) \
            .set({'year': year, 'system:time_start':date_start.millis()}) \
            .set({'year': year, 'system:time_end':date_end.millis()})
                
    return(fraction)


# Generate a list of Years based on the years of L5 & L8 timeseries
year_list = [x for x in range(ee.Date(DATES_LS5[0]).get('year').getInfo(), ee.Date(DATES_LS8[1]).get('year').getInfo())]
_ = [year_list.remove(y) for y in [2012,2013]]

# apply Unmix function for each year in image collection 
UnmixCol =  ee.ImageCollection(ee.List(year_list).map(func_Unmix))